In [14]:
import mygene
import scanpy as sc, scprep
from scipy.sparse import csr_matrix
import pandas as pd
import numpy as np

In [5]:
path_ds = 'raw/'
path_out ='processed/'

In [6]:
! wget ftp://ftp.ncbi.nlm.nih.gov/geo/series/GSE81nnn/GSE81608/matrix/GSE81608_series_matrix.txt.gz
! mv GSE81608_series_matrix.txt.gz raw

! wget ftp://ftp.ncbi.nlm.nih.gov/geo/series/GSE81nnn/GSE81608/suppl/GSE81608_human_islets_rpkm.txt.gz
! mv GSE81608_human_islets_rpkm.txt.gz raw

--2024-03-05 19:40:43--  ftp://ftp.ncbi.nlm.nih.gov/geo/series/GSE81nnn/GSE81608/matrix/GSE81608_series_matrix.txt.gz
           => ‘GSE81608_series_matrix.txt.gz’
Resolving ftp.ncbi.nlm.nih.gov (ftp.ncbi.nlm.nih.gov)... 130.14.250.10, 130.14.250.7, 2607:f220:41e:250::12, ...
Connecting to ftp.ncbi.nlm.nih.gov (ftp.ncbi.nlm.nih.gov)|130.14.250.10|:21... connected.
Logging in as anonymous ... Logged in!
==> SYST ... done.    ==> PWD ... done.
==> TYPE I ... done.  ==> CWD (1) /geo/series/GSE81nnn/GSE81608/matrix ... done.
==> SIZE GSE81608_series_matrix.txt.gz ... 44214
==> PASV ... done.    ==> RETR GSE81608_series_matrix.txt.gz ... done.
Length: 44214 (43K) (unauthoritative)

GSE81608_series_mat 100%[===================>]  43.18K  --.-KB/s    in 0.04s   

2024-03-05 19:40:43 (1.18 MB/s) - ‘GSE81608_series_matrix.txt.gz’ saved [44214]

--2024-03-05 19:40:43--  ftp://ftp.ncbi.nlm.nih.gov/geo/series/GSE81nnn/GSE81608/suppl/GSE81608_human_islets_rpkm.txt.gz
           => ‘GSE81608_human_i

In [11]:
obs=pd.read_table(path_ds+'GSE81608_series_matrix.txt.gz',skiprows=26,index_col=0)
x=pd.read_table(path_ds+'GSE81608_human_islets_rpkm.txt.gz', index_col=0)

In [12]:
mg = mygene.MyGeneInfo()
genemap = mg.querymany(x.index.to_list(), scopes='entrezgene', fields=['ensembl.gene','symbol'], species='human')

1857 input query terms found no hit:	['244', '2748', '3262', '9142', '9220', '9503', '10638', '10896', '11217', '23285', '25787', '26148'


In [15]:
genemap_df=[]
for g in genemap:
    g_parsed={'uid':g['query']}
    g_parsed['gene_symbol']=g['symbol'] if 'symbol' in g else np.nan
    # Genes with multiple EIDs have these as list
    if 'ensembl' in g:
        if isinstance(g['ensembl'],list):
            g_parsed['EID']=','.join([gs['gene'] for gs in g['ensembl']])
        else:
            g_parsed['EID']=g['ensembl']['gene']
    genemap_df.append(g_parsed)
genemap_df=pd.DataFrame(genemap_df)
genemap_df.index=genemap_df.uid
genemap_df.drop('uid',axis=1,inplace=True)

In [16]:
adata=sc.AnnData(X=csr_matrix(x.T),obs=pd.DataFrame(index=x.columns),
                 var=pd.DataFrame(index=x.index))

/tmp/ipykernel_3590210/1645750452.py:1: FutureWarning: X.dtype being converted to np.float32 from float64. In the next version of anndata (0.9) conversion will not be automatic. Pass dtype explicitly to avoid this warning. Pass `AnnData(X, dtype=X.dtype, ...)` to get the future behavour.
  adata=sc.AnnData(X=csr_matrix(x.T),obs=pd.DataFrame(index=x.columns),
/gpfs/gibbs/project/krishnaswamy_smita/av622/conda_envs/gspa/lib/python3.8/site-packages/anndata/_core/anndata.py:121: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)


In [17]:
# Sqrt transform
adata.layers['normalised_original']=adata.X.copy()
sc.pp.sqrt(adata)

In [18]:
# Add gene info
for col in genemap_df:
    adata.var[col]=genemap_df[col]

In [19]:
# Parse obs
obs=obs.T[['!Sample_geo_accession','!Sample_characteristics_ch1']]
obs.columns=['geo_accession','tissue','donor','disease','age',
             'ethnicity','sex','cell_type_original']

In [20]:
# Parse obs
obs.drop('tissue',axis=1,inplace=True)
obs['donor']=obs['donor'].str.replace('donor id: ','')
obs['disease']=obs['disease'].str.replace('condition: ','').str.replace('non-diabetic','control')
obs['age']=obs['age'].apply(lambda x: x.replace('age: ','')+' y')
obs['ethnicity']=obs['ethnicity'].str.replace('ethnicity: ','').map(
    {'AA':'african_american','C':'caucasian','AI':'asian_indian','H':'hispanic'})
obs['sex']=obs['sex'].str.replace('gender: ','').map({'M':'male','F':'female'})
obs['cell_type_original']=obs['cell_type_original'].str.replace('cell subtype: ','')
obs['cell_type']=obs['cell_type_original'].replace({'PP':'gamma'})

In [24]:
donor_df=pd.read_csv(path_ds+'GSE81608_donors.txt',index_col=0, sep=' ')

In [51]:
donor_df.index = [x + ' ' + str(donor_df.iloc[i]['ID']) for i,x in enumerate(donor_df.index)]

In [53]:
for col in ['BMI','HbA1c']:
    obs[col]=obs.donor.map(donor_df[col].to_dict())

In [54]:
obs.index=[i.replace('Pancreatic islet cell sample ','Sample_') for i in obs.index]

In [56]:
adata.obs=obs.reindex(adata.obs_names)

In [60]:
adata_subset = adata[(adata.obs['disease'].isin(['control','T2D'])) & (adata.obs['cell_type'] == 'beta')]

In [67]:
adata_subset.write(path_out+'GSE81608_adata.h5ad')

/gpfs/gibbs/project/krishnaswamy_smita/av622/conda_envs/gspa/lib/python3.8/site-packages/anndata/_core/anndata.py:1235: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  df[key] = c
/gpfs/gibbs/project/krishnaswamy_smita/av622/conda_envs/gspa/lib/python3.8/site-packages/anndata/_core/anndata.py:1235: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  df[key] = c
/gpfs/gibbs/project/krishnaswamy_smita/av622/conda_envs/gspa/lib/python3.8/site-packages/anndata/_core/anndata.py:1235: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  df[key] = c
/gpfs/gibbs/project/krishnaswamy_smita/av622/conda_envs/gspa/lib/python3.8/site-packages/anndata/_core/anndata.py:1235: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  df[key] = c
/gpfs/gibbs/project/krishnaswamy_smita/av622/conda_envs/gspa